# Exercise Notebook on Interacting with Databases: Solutions

In [1]:
import pandas as pd
import numpy as np
import os.path

In this exercise notebook we will work with a SQLite database. First, let's create a new database from the *movielens* dataset. The ratings data are very large, ingesting those into SQLite is quite slow. Let's only take the first 100 thousand records.

In [2]:
# DO NOT MODIFY

# set here the relative path to the movielens folder
MOVIELENS="../../../movielens"

movies = pd.read_csv(os.path.join(MOVIELENS, 'movies.csv'), sep=',')
ratings = pd.read_csv(os.path.join(MOVIELENS, 'ratings.csv'), sep=',')

In [3]:
# DO NOT MODIFY

ratings = ratings.iloc[:100000]

We first create on disk the SQLite databse `movielens.db` and then we use the `to_sql` method of DataFrames to store the data there, we specify that is the table exists already, it should be replaced.

In [4]:
# DO NOT MODIFY

import sqlite3
conn = sqlite3.connect("movielens.db")

In [5]:
print("Size of the database is {:.2g} MB".format(os.path.getsize("movielens.db")/1024**2))

Size of the database is 0 MB


In [6]:
# DO NOT MODIFY

movies.to_sql("movies", conn, if_exists="replace")

In [7]:
# DO NOT MODIFY

ratings.to_sql("ratings", conn, if_exists="replace")

In [8]:
print("Size of the database is {:.2g} MB".format(os.path.getsize("movielens.db")/1024**2))

Size of the database is 5.5 MB


## Exercise 1: get the names of all the tables in the database 

First let's find out the name of the tables available inside the database

In [9]:
cursor = conn.cursor()

In [10]:
def get_list_of_tables(sqlite_cursor):
    """Return a list of table names in a sqlite database
    
    Parameters
    ----------
    sqlite_cursor : sqlite3.Cursor
    SQLite database cursor
    
    Returns
    -------
    tables : list of strings
    list of table names"""
### BEGIN SOLUTION
    tables = []
    for row in sqlite_cursor.execute("SELECT name FROM sqlite_master"):
        tables.append(row[0])
    return tables
### END SOLUTION

In [11]:
movielens_db_tables = get_list_of_tables(cursor)
movielens_db_tables

['movies', 'ix_movies_index', 'ratings', 'ix_ratings_index']

In [12]:
# DO NOT MODIFY

assert isinstance(movielens_db_tables[0], str), "Make sure that the function does not return a list of tuples but just a list of strings"
assert "movies" in movielens_db_tables, "It looks like the movies table is missing"
assert "ratings" in movielens_db_tables, "It looks like the ratings table is missing"

## Exercise 2: Print a few ratings records

Let's use the `sqlite3` package to extract the first 3 ratings from the ratings table.

In [13]:
first_3_ratings = None
### BEGIN SOLUTION
first_3_ratings = cursor.execute("SELECT * FROM ratings LIMIT 3").fetchall()
### END SOLUTION

In [14]:
first_3_ratings

[(0, 1, 2, 3.5, 1112486027),
 (1, 1, 29, 3.5, 1112484676),
 (2, 1, 32, 3.5, 1112484819)]

In [15]:
assert len(first_3_ratings) == 3, "Make sure you are selecting only 3 rows"
assert len(first_3_ratings[0]) == 5, "Something went wrong, each row of ratings should have 5 columns"

Explore more the first_3_ratings data structure, what kind of Python object is it? Find out with `type()`.
What kind of Python object is each element of it?

## Exercise 3: Load the ratings table in Pandas

Let's load the entire ratings table into a dataframe

In [16]:
ratings_dataframe = None
### BEGIN SOLUTION
ratings_dataframe = pd.read_sql_query("SELECT * FROM ratings", conn)
### END SOLUTION

In [17]:
assert len(ratings_dataframe) == 100000, "Check your code, you should have 100000 rows"

## Exercise 4: Load a subset of the movies table in Pandas

We would like now to only load the movies of the Fantasy genre into a Pandas dataframe, not the complete movies table. This is very useful for large datasets because we can save a lot of memory if you avoid doing the filtering in pandas but let the database take care of it.

Let's first introduce the SQL syntax for checking if a records contains a string, for example let's print the first 3 Children movies:

In [18]:
cursor.execute("SELECT * FROM movies WHERE genres LIKE '%Children%' LIMIT 3").fetchall()

[(0, 1, 'Toy Story (1995)', 'Adventure|Animation|Children|Comedy|Fantasy'),
 (1, 2, 'Jumanji (1995)', 'Adventure|Children|Fantasy'),
 (7, 8, 'Tom and Huck (1995)', 'Adventure|Children')]

In [19]:
fantasy_movies_dataframe = None
### BEGIN SOLUTION
fantasy_movies_dataframe = pd.read_sql_query("SELECT * FROM movies WHERE genres LIKE '%Fantasy%'", conn)
### END SOLUTION

In [20]:
# DO NOT MODIFY

assert len(fantasy_movies_dataframe) == 1412, "Make sure you do not have LIMIT in the SQL query"
assert fantasy_movies_dataframe.genres.str.contains("Fantasy").all(), "All rows should contain the Fantasy string, try to identify which rows are not"

In the previous cell, we are using the `.all()` DataFrame method, what is it doing? look for its documentation and play with it!